In [3]:
import pandas as pd
import numpy as np
training_df = pd.read_csv('train.csv')
testing_df = pd.read_csv('test.csv')

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Assuming the first column is 'id' and the following columns are the features,
# and the last 12 columns are the 6 traits mean and sd
# Features selection: Exclude the 'id' column and the target columns (mean and sd values of the traits)
X = training_df.drop(columns=['id', 'X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean', 'X4_sd', 'X11_sd', 'X18_sd', 'X26_sd', 'X50_sd', 'X3112_sd'])

# Targets selection: Only the mean values of the traits
y_means = training_df[['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']]


# Splitting the data into training and validation sets for each trait
for column in y_means.columns:
    y = y_means[column]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the RandomForestRegressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the validation set
    predictions = model.predict(X_val)

    # Evaluate the model
    mse = mean_squared_error(y_val, predictions)
    rmse = np.sqrt(mse)
    print(f"RMSE for {column}: {rmse}")

# Adjust the code for your actual file paths and column indices as necessary.


RMSE for X4_mean: 0.16192624201910594
RMSE for X11_mean: 15177.763409490393
RMSE for X18_mean: 3191701.3825716935
RMSE for X26_mean: 323695.84244395595
RMSE for X50_mean: 1565.3173075890318
RMSE for X3112_mean: 204648663.06719327


**The code above ran for 4 and a half hours on my computer, but it rewrote the models over each other, so we cant use it to make predictions**

In [5]:

# Prepare the features from the test set, matching the training feature selection
X_test = testing_df.drop(columns=['id'])

# Initialize a dictionary to store predictions
predictions = {}

# Assuming your models are stored in a dictionary models with keys as trait names
# e.g., models = {'X4_mean': model_X4, ...}
for trait in ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']:
    # Predict using each model and store the predictions
    predictions[trait] = model[trait].predict(X_test)

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'id': testing_df['id'],
    **predictions  # This unpacks the predictions dict into the DataFrame
})

# Ensure the column order matches the sample_submission.csv, adjust if necessary
submission_df = submission_df[['id', 'X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']]

# Save the submission file
submission_df.to_csv('Random Forest Attempt 1', index=False)


TypeError: list indices must be integers or slices, not str

In [6]:
from joblib import dump

# After each model is trained within the loop:
dump(model, f'model_{column}.joblib')
from joblib import load

model_X4 = load('model_X4.joblib')
# Repeat for other models


FileNotFoundError: [Errno 2] No such file or directory: 'model_X4.joblib'